- 신경망은 데이터에 대한 작업을 수행하는 계층/모듈로 구성된다.
- torch.nn 네임 스페이스는 자신의 신경망을 구축하는 데 필요한 모든 빌딩 블록을 제공합니다.
- Every module in PyTorch subclasses the nn.Module.
- 신경망은 다른 모듈(레이어)로 구성된 모듈 그 자체이다.
- 이러한 중첩된 구조를 통해 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [3]:
cuda_obj = torch.cuda

In [4]:
dir(cuda_obj)

['Any',
 'BFloat16Storage',
 'BFloat16Tensor',
 'BoolStorage',
 'BoolTensor',
 'ByteStorage',
 'ByteTensor',
 'CharStorage',
 'CharTensor',
 'ComplexDoubleStorage',
 'ComplexFloatStorage',
 'CudaError',
 'DeferredCudaCallError',
 'Device',
 'Dict',
 'DoubleStorage',
 'DoubleTensor',
 'Event',
 'FloatStorage',
 'FloatTensor',
 'HalfStorage',
 'HalfTensor',
 'IntStorage',
 'IntTensor',
 'List',
 'LongStorage',
 'LongTensor',
 'Optional',
 'ShortStorage',
 'ShortTensor',
 'Stream',
 'Tuple',
 'Union',
 '_CudaBase',
 '_CudaDeviceProperties',
 '_Graph',
 '_StorageBase',
 '__annotations__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_check_capability',
 '_check_cubins',
 '_cudart',
 '_device',
 '_device_t',
 '_dummy_type',
 '_get_device_index',
 '_initialization_lock',
 '_initialized',
 '_is_in_bad_fork',
 '_lazy_call',
 '_lazy_init',
 '_lazy_new',
 '_queued_calls',
 '_sleep',
 '_tls',
 '_utils',
 'amp',
 '

In [5]:
cuda_obj.list_gpu_processes()
## pip install pynvml

'GPU:0\nno processes are running'

In [6]:
cuda_obj.list_gpu_processes()

'GPU:0\nno processes are running'

In [7]:
cuda_obj.device_count()

2

In [8]:
cuda_obj.get_device_name()

'GeForce RTX 3090'

In [9]:
cuda_obj.get_device_name('cuda:0')

'GeForce RTX 3090'

In [10]:
cuda_obj.get_device_name('cuda:1')

'GeForce RTX 3060'

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [12]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda:0 device


In [13]:
torch.cuda.set_device('cuda:0')

In [14]:
cuda_obj.get_device_name()

'GeForce RTX 3090'

In [15]:
torch.cuda.set_device('cuda:1')

In [16]:
cuda_obj.get_device_name()

'GeForce RTX 3060'

#### Define the Class

In [17]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [18]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [19]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


In [20]:
#### Model Layers
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [21]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [22]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [23]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1586, -0.2399, -0.3205, -0.4057, -0.2438, -0.4419, -0.2187,  0.4612,
          0.1095,  0.0542, -0.2630, -0.0433,  0.1428, -0.2556, -0.1149,  0.3500,
         -0.3799, -0.1457,  0.0826, -0.6201],
        [-0.1320, -0.1106,  0.1164, -0.1295, -0.2179, -0.1809, -0.2783,  0.1757,
          0.1932,  0.2960,  0.1485,  0.0652,  0.0202,  0.0042, -0.3440,  0.4649,
          0.1074, -0.2309,  0.2906, -0.3376],
        [ 0.2383,  0.0151, -0.3839, -0.1730, -0.2956, -0.5519, -0.2498,  0.1869,
          0.1992,  0.3229,  0.0347,  0.3011, -0.1997, -0.0168, -0.1435,  0.3267,
         -0.0894, -0.3672,  0.0442, -0.2774]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.1586, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4612, 0.1095,
         0.0542, 0.0000, 0.0000, 0.1428, 0.0000, 0.0000, 0.3500, 0.0000, 0.0000,
         0.0826, 0.0000],
        [0.0000, 0.0000, 0.1164, 0.0000, 0.0000, 0.0000, 0.0000, 0.1757, 0.1932,
         0.2960, 0.1485, 0.0652, 0.0202, 0.0042, 0.000

- nn.Sequential은 모듈의 정렬된 컨테이너입니다. 데이터는 정의된 것과 동일한 순서로 모든 모듈을 통과합니다. 순차 컨테이너를 사용하여 seq_modules와 같은 빠른 네트워크를 구성할 수 있습니다.

In [24]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

- 신경망의 마지막 선형 레이어는 nn으로 전달되는 로짓([-infty, infty]의 원시 값)을 반환합니다.소프트맥스 모듈. 로짓은 각 클래스에 대한 모형의 예측 확률을 나타내는 값[0, 1]으로 척도화됩니다. dim 매개 변수는 값이 1이 되어야 하는 차원을 나타냅니다

In [25]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.1247, 0.1029, 0.0778, 0.1058, 0.1054, 0.0879, 0.1248, 0.0768, 0.0864,
         0.1076],
        [0.1159, 0.0958, 0.0806, 0.1094, 0.1149, 0.0940, 0.1275, 0.0816, 0.0734,
         0.1069],
        [0.1091, 0.0980, 0.0880, 0.1083, 0.1074, 0.0903, 0.1181, 0.0834, 0.0842,
         0.1132]], grad_fn=<SoftmaxBackward>)

#### Model Parameters

- 신경망 내부의 많은 계층은 매개 변수화된다. 즉, 훈련 중에 최적화된 관련 가중치와 편향을 가지고 있다. 하위 분류 n.모듈은 모델 객체 내부에 정의된 모든 필드를 자동으로 추적하고 모델의 매개변수() 또는 named_parameters() 방법을 사용하여 모든 매개변수에 액세스할 수 있도록 합니다.

In [26]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0103,  0.0097, -0.0116,  ..., -0.0295,  0.0299, -0.0343],
        [ 0.0320,  0.0030,  0.0079,  ..., -0.0068, -0.0049,  0.0316]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0254,  0.0159], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0250,  0.0111, -0.0247,  ...,  0.0314, -0.0107,  0.0110],
        [-0.0021, -0.0035,  0.0362,  ...,  0.0081,  0.0074, -0.0412]],
       device='cuda:0', grad_fn=<Sl

#### AUTOMATIC DIFFERENTIATION WITH TORCH.AUTOGRAD

- 신경망을 훈련할 때 가장 자주 사용되는 알고리즘은 후방 전파다. 이 알고리즘에서 매개 변수(모델 가중치)는 주어진 매개 변수에 대해 손실 함수의 기울기에 따라 조정된다.
- 이러한 구배를 계산하기 위해, 파이토치는 토치라고 불리는 내장된 차별화 엔진을 가지고 있다.오토그라드 그것은 모든 계산 그래프에 대한 그라데이션의 자동 계산을 지원한다.
- 입력 x, 매개 변수 w와 b, 그리고 일부 손실 함수를 가진 가장 단순한 단일 계층 신경망을 고려해보자. 다음과 같은 방법으로 PyTorch에서 정의할 수 있다.

In [27]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [28]:
loss

tensor(1.3783, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

#### Tensors, Functions and Computational graph

![Computational_Graph](./images/image01.png)

- 이 네트워크에서 w와 b는 매개 변수이며, 최적화해야 합니다. 따라서, 우리는 그러한 변수들에 대해 손실 함수의 기울기를 계산할 수 있어야 한다. 이를 위해, 우리는 텐서의 required_grad 속성을 설정한다.
- 텐서를 생성할 때 또는 나중에 x.requires_grad_(True) 방법을 사용하여 requires_grad 값을 설정할 수 있습니다.
- 계산 그래프를 구성하기 위해 텐서에 적용하는 함수는 사실 클래스 함수의 객체이다. 이 객체는 순방향으로 함수를 계산하는 방법과 역방향 전파 단계 동안 함수의 도함수를 계산하는 방법을 알고 있습니다.
- 역방향 전파 함수에 대한 참조는 텐서의 grad_fn 속성에 저장된다. 기능에 대한 자세한 내용은 설명서를 참조하십시오.

In [29]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f4243b2c210>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7f40f494c850>


#### Computing Gradients


- 신경망에서 매개 변수의 가중치를 최적화하려면 매개 변수에 대한 손실 함수의 도함수를 계산해야 한다
- 즉, \frac{\partial loss}{\partial w}가 필요하다. 
- ww loss​및 \fracsible loss}{\fracs b} bb loss​x와 y의 일부 고정 값 하에서. 이러한 도함수를 계산하기 위해, 우리는 loss.backward()라고 부르고, w.grad와 b.grad에서 값을 검색한다.

In [30]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1337, 0.3231, 0.0441],
        [0.1337, 0.3231, 0.0441],
        [0.1337, 0.3231, 0.0441],
        [0.1337, 0.3231, 0.0441],
        [0.1337, 0.3231, 0.0441]])
tensor([0.1337, 0.3231, 0.0441])


- 계산 그래프의 리프 노드에 대한 grad 속성만 얻을 수 있으며, required_grad 속성이 True로 설정되어 있다. 그래프의 다른 모든 노드의 경우 그레이디언트를 사용할 수 없다.
- 성능상의 이유로 주어진 그래프에서 뒤로 한 번만 사용하여 그레이디언트 계산을 수행할 수 있다. 동일한 그래프에서 여러 개의 역방향 호출을 수행해야 하는 경우 retain_graph=True를 역방향 호출로 전달해야 합니다.

#### Disabling Gradient Tracking

- 기본적으로 required_grad=True인 모든 텐서는 계산 기록을 추적하고 구배 계산을 지원합니다.
- 그러나, 예를 들어 모델을 훈련하고 일부 입력 데이터에 적용하려고 할 때, 즉 네트워크를 통해 전진 계산만 수행하고자 할 때, 그렇게 할 필요가 없는 경우가 있다. 계산 코드를 torch.no_gradsec 블록으로 둘러서 계산 추적을 중지할 수 있다.

In [31]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

True


In [32]:
with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

False


In [33]:
z

tensor([-0.4009,  3.4491, -1.8808])

- 동일한 결과를 얻는 또 다른 방법은 텐서에 분리() 방법을 사용하는 것이다.

In [34]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


- 그라데이션 추적을 비활성화할 수 있는 이유는 다음과 같습니다.
  - 신경망의 일부 파라미터를 고정된 파라미터로 표시합니다. 이것은 사전 훈련된 네트워크를 미세 조정하는 매우 일반적인 시나리오이다.
  - 그레이디언트를 추적하지 않는 텐서의 계산이 더 효율적이기 때문에 전진 패스만 할 때 계산 속도를 높입니다.



#### More on Computational Graphs


- 개념적으로, autograd는 Function 객체로 구성된 DAG(Directed Acyclic Graph)에 데이터(텐서) 및 모든 실행 작업(결과적인 새 텐서) 기록을 유지한다. 이 DAG에서 리프는 입력 텐서이고 루트는 출력 텐서이다. 이 그래프를 뿌리부터 잎까지 추적하면 체인 규칙을 사용하여 자동으로 그라데이션이 계산됩니다.

- 전진 패스에서 오토그라드는 두 가지 작업을 동시에 수행합니다.
  - 결과 텐서를 계산하기 위해 요청된 작업을 실행합니다.
  - DAG에서 작업의 그라데이션 기능을 유지합니다.
- DAG 루트에서 .backward()가 호출되면 백워드 패스가 시작됩니다. 그러면 autograd:
  - 각 .grad_fn에서 구배를 계산합니다
  - 각 텐서의 .grad 속성에 누적한다.
  - 체인 규칙을 사용하면 리프 텐서까지 전파됩니다.

#### Optional Reading: Tensor Gradients and Jacobian Products

- 많은 경우, 우리는 스칼라 손실 함수를 가지고 있으며, 우리는 일부 매개 변수에 대해 기울기를 계산해야 한다. 그러나 출력 함수가 임의의 텐서인 경우가 있다. 이 경우 PyTorch를 사용하면 실제 기울기가 아니라 소위 야코비안 곱(https://www.youtube.com/watch?v=XF44_HAMnKY)을 계산할 수 있습니다 
- Jacobian 행렬 자체를 계산하는 대신 PyTorch를 사용하면 Jacobian 곱 v^T\cdot Jv를 계산할 수 있습니다. t 주어진 입력 벡터 v=(v_1 \dump v_m)v=(v)에 대한 ΔJ 1​…v m​).
- 이는 vv를 인수로 하여 역방향으로 호출함으로써 달성된다. vv의 크기는 곱을 계산하고자 하는 원래 텐서의 크기와 같아야 한다.

In [35]:
inp = torch.eye(5, requires_grad=True)
inp

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], requires_grad=True)

In [36]:
out = (inp+1).pow(2)
out

tensor([[4., 1., 1., 1., 1.],
        [1., 4., 1., 1., 1.],
        [1., 1., 4., 1., 1.],
        [1., 1., 1., 4., 1.],
        [1., 1., 1., 1., 4.]], grad_fn=<PowBackward0>)

In [37]:
out.backward(torch.ones_like(inp), retain_graph=True)
out

tensor([[4., 1., 1., 1., 1.],
        [1., 4., 1., 1., 1.],
        [1., 1., 4., 1., 1.],
        [1., 1., 1., 4., 1.],
        [1., 1., 1., 1., 4.]], grad_fn=<PowBackward0>)

In [38]:
print("First call\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


In [39]:
out.backward(torch.ones_like(inp), retain_graph=True)
out

tensor([[4., 1., 1., 1., 1.],
        [1., 4., 1., 1., 1.],
        [1., 1., 4., 1., 1.],
        [1., 1., 1., 4., 1.],
        [1., 1., 1., 1., 4.]], grad_fn=<PowBackward0>)

In [40]:
print("\nSecond call\n", inp.grad)


Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])


In [41]:
inp.grad.zero_()
inp

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], requires_grad=True)

In [42]:
out.backward(torch.ones_like(inp), retain_graph=True)
out

tensor([[4., 1., 1., 1., 1.],
        [1., 4., 1., 1., 1.],
        [1., 1., 4., 1., 1.],
        [1., 1., 1., 4., 1.],
        [1., 1., 1., 1., 4.]], grad_fn=<PowBackward0>)

In [43]:
print("\nCall after zeroing gradients\n", inp.grad)


Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


- 같은 인수를 사용하여 두 번째로 뒤로 호출할 때 그라디언트 값이 다릅니다. 이것은 역방향 전파를 수행할 때 PyTorch가 그라디언트를 축적하기 때문에 발생합니다. 계산된 그래디언트의 값은 계산 그래프의 모든 리프 노드의 그래드 특성에 추가됩니다. 적절한 그라데이션을 계산하려면 이전에 그라데이션 속성을 제로화해야 합니다. 실제 훈련에서 옵티마이저는 우리가 이것을 할 수 있도록 도와줍니다.
- 이전에는 매개 변수 없이 역방향() 함수를 호출했습니다. 이는 본질적으로 신경망 훈련 중 손실과 같은 스칼라 값 함수의 경우 기울기를 계산하는 유용한 방법인 백워드(토치.텐서(1.0) 호출과 동일하다.